In [1]:
import numpy as np
import pandas as pd
import warnings

from gensim.models.doc2vec import Doc2Vec
from pandarallel import pandarallel
from tqdm import tqdm

from src.models import cf

pandarallel.initialize()
tqdm.pandas()
warnings.filterwarnings('ignore')

/Users/jensen/Desktop/recommendation-systems/projects/product-reviews-recommender/.venv/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


# Load Data and Models

In [2]:
# global variables
DATA_PATH = "data/evaluation"
D2V_PATH = "models/d2v"
CATEGORY = "Grocery_and_Gourmet_Food"

train = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_train.csv")
d2v = Doc2Vec.load(f"{D2V_PATH}/{CATEGORY}_50_d2v.model")

In [3]:
# checking train dataframe
train.head().append(train.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,0,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A23RYWDS884TUL,5.0,This curry paste makes a delicious curry. I j...,2013-05-28,curry paste delicious curry fry chicken vegeta...
1,1,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A945RBQWGZXCK,5.0,I've purchased different curries in the grocer...,2012-09-17,purchase different curry grocery store complet...
2,3,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A3AMNY44OP8AOU,4.0,I started a new diet restricting all added sug...,2014-01-23,start new diet restrict added sugar brand suga...
3,4,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A3IB4CQ2QEJLJ8,5.0,So many flavors. I can't begin to tell you how...,2014-04-27,flavor begin tell love mae ploy curry ask reci...
4,5,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",AQA5DF3RWKETQ,5.0,I've used this a lot recently in some of my ch...,2012-11-27,use lot recently chicken dish use lot like spi...
47769,77420,B00I33696K,Reese's Miniature Peanut Butter Cups .31oz - 1...,"['Grocery & Gourmet Food', 'Candy & Chocolate'...",A192LQZWDYPR4U,5.0,Another quality Reese Peanut Butter Cup produc...,2014-02-27,quality reese peanut butter cup product great ...
47770,77421,B00I33696K,Reese's Miniature Peanut Butter Cups .31oz - 1...,"['Grocery & Gourmet Food', 'Candy & Chocolate'...",A2QKXW3LDQ66P5,5.0,I purchased these for my husband who has every...,2013-02-20,purchase husband love reeses valentine day pre...
47771,77430,B00ID9VSOM,"Viva Labs Organic Coconut Sugar: Non-GMO, Low-...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A2P3TGJU301KXD,5.0,this stuff is INCREDIBILY yummy! SO much bette...,2014-07-15,stuff incredibily yummy good regular brown sug...
47772,77456,B00IRL93SY,Barrie House Kenya Estate - AA Single Cup Caps...,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",AEFE9VDHTQ199,5.0,"Very nice aroma, body and taste! Will buy this...",2014-05-24,nice aroma body taste buy coffee good coffee a...
47773,77508,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A2AEZQ3DGBBLPR,2.0,This is a no go for diabetics according to my ...,2014-06-26,diabetic accord wife doctor order intention us...


In [4]:
# testing d2v models
d2v.dv[0]

array([-2.0247097 , -1.3294973 , -0.21396987,  0.4237852 ,  0.00897375,
        2.1241405 ,  0.04780632,  1.0691372 , -0.11706699, -0.46099463,
       -0.08401425, -1.0733169 , -0.21556404,  0.08397805, -0.025368  ,
       -1.5263675 ,  0.11098978,  0.41218644, -0.01474508, -1.0207732 ,
       -0.67836535,  0.21077251, -0.51048225, -0.1605686 ,  0.13394798,
        1.6452312 , -1.1992716 ,  1.9470034 , -2.037423  ,  0.868756  ,
        0.8278804 ,  0.70229375,  0.63159204,  1.3180909 ,  0.36948895,
       -0.55629754, -1.2663819 , -1.5738735 , -1.4281844 ,  1.90474   ,
       -0.5490278 , -0.90059054, -1.3286449 , -0.9170224 ,  0.06542239,
        0.00427322, -0.6460078 , -0.2183445 ,  1.2150261 ,  0.9847518 ],
      dtype=float32)

# Generate User Embeddings

In [5]:
# get user rating history
train_user_rating_history = train.groupby(["reviewerID"])["asin"].progress_apply(list)
print(train_user_rating_history)

100%|██████████████████████████████████████████████████████| 13397/13397 [00:00<00:00, 61740.17it/s]

reviewerID
A00177463W0XWB16A9O05                             [B0029XDZIK, B0094ISOMA]
A022899328A0QROR32DCT                             [B001ACMCNU, B003TO9RSU]
A068255029AHTHDXZURNU                             [B000K8WVYA, B0094ISOMA]
A06944662TFWOKKV4GJKX                                         [B000CQBZPG]
A1004703RC79J9                                                [B001E50THY]
                                               ...                        
AZWRZZAMX90VT            [B0007R9L5Q, B000CQ01GU, B000E123IC, B000E46LZ...
AZXKAH2DE6C8A            [B000EML7DS, B000ODF2ME, B001650XUK, B0018QLG9...
AZXON596A1VXC                         [B00113SKZW, B00113ZTVK, B001L4JH5I]
AZYXC63SS008M                                                 [B0040WCQKQ]
AZZ5ASC403N74                                                 [B004U49QU2]
Name: asin, Length: 13397, dtype: object


In [6]:
def generate_user_embeddings(user_rating_history: pd.DataFrame, d2v: Doc2Vec) -> dict:
    """
    
    Args:
        unique_users ([list]):
        d2v ([Doc2Vec]):
    """
    
    # generate unique users
    unique_users = user_rating_history.reset_index()["reviewerID"].tolist()
    
    user_embeddings = {}
    for user in tqdm(unique_users):
        user_embedding = np.zeros(50)
        for item in user_rating_history[user]:
            user_embedding += d2v.dv[item]
            
        # computing mean aggregation
        user_embedding /= len(user_rating_history[user])
        user_embeddings[user] = user_embedding
        
    return user_embeddings

In [7]:
train_user_embeddings = generate_user_embeddings(train_user_rating_history, d2v)

100%|██████████████████████████████████████████████████████| 13397/13397 [00:00<00:00, 37724.29it/s]


# Utility Functions

In [8]:
def get_top_n(predictions: dict, user_rating_history: pd.DataFrame, n=10) -> dict:
    """Return the top-N recommendations for each user based on cosine similarity.
    
    Args:
        d2v ([Doc2Vec]): Doc2Vec of item representations based on reviews.
        user_rating_history ([dict]): "Future" purchases aggregated by users based on
            test set.
    
    Returns:
        ([dict]): A dictionary of top-N recommendations for each unique user, sorted by
            cosine similarties.
    """
    
    # retrieve a 200 items candidate list based on similarities
    top_ns = {}
    for user in predictions:
        rated_items = user_rating_history[user]
        candidate_items = [i[0] for i in predictions[user]]
        unrated_items = set(candidate_items) - set(rated_items)
        
        user_top_n = []
        idx = 0
        while len(user_top_n) < n:
            if candidate_items[idx] in unrated_items:
                user_top_n.append(candidate_items[idx])
                idx += 1
            else:
                idx += 1
        
        top_ns[user] = user_top_n
        
    return top_ns

def recall_at_k(asins, predicted_asins, k=10):
    # number of relevant items
    set_actual = set(asins)
    set_preds = set(predicted_asins)
    num_relevant = len(set_actual.intersection(set_preds))
    
    # calculating recall@K - relevant / total relevant items
    recall_at_k = num_relevant / len(asins)
    
    return recall_at_k

def novelty_at_k(item_popularity, predicted_asins, k=10):
    """
    """
    # finding avg novelty
    popularity_sum = item_popularity.loc[predicted_asins].sum()
    novelty_at_k = ((k*1) - popularity_sum) / k
    
    return novelty_at_k

def generate_item_popularity(train: pd.DataFrame) -> pd.DataFrame:
    """
    """
    
    # create a mapping of item popularatity
    # based on sum(item's review / max reviews) / no items
    max_reviews = (train.groupby(['asin'])
                   .agg({'processedReviewText': 'count'})
                   .max()
                   .values[0])
    item_popularity = (train.groupby(['asin'])
                       .agg({'processedReviewText': 'count'})
                       .apply(lambda x: x/max_reviews))
    
    return item_popularity
    

def evaluate_recommendations(top_ns: dict, user_rating_history: pd.DataFrame, item_popularity: pd.DataFrame, k=10) -> pd.DataFrame:
    """
    
    Args:
        top_ns
        user_rating_history
    """
    
    test_recommendations = pd.DataFrame(top_ns.items(), columns=["reviewerID", "pred_asin"])
    
    # combined test history and recommendations
    test_merged = pd.merge(user_rating_history, test_recommendations, on="reviewerID", how="inner")
    
    # generating recall@k metrics
    test_merged["recall@k"] = test_merged.apply(lambda x: recall_at_k(x.asin, x.pred_asin, k=k), axis=1)
    test_merged["novelty@k"] = test_merged.apply(lambda x: novelty_at_k(item_popularity, x.pred_asin, k=k), axis=1)
    average_recall_at_k = test_merged["recall@k"].mean()
    average_novelty_at_k = test_merged["novelty@k"].mean()
    
    print(f"The MEM-ECF has an average recall@{k}: {average_recall_at_k:.5f}, average novelty@{k}: {average_novelty_at_k:.5f}")
    
    return test_merged

# Generate N-Recommendations = {10, 25, 30, 45}

## Load Test Data

In [9]:
test = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_test.csv")

In [10]:
test.head().append(test.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,2,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A1TCSC0YWT82Q0,5.0,I love ethnic foods and to cook them. I recent...,2013-08-03,love ethnic food cook recently purchase produc...
1,8,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A1Z7Y2GMAP9SRY,5.0,I like to make my own curry but this is a tast...,2014-06-27,like curry tasty alternative use base kind dif...
2,23,B00004S1C5,"Ateco Food Coloring Kit, 6 colors","['Grocery & Gourmet Food', 'Cooking & Baking',...",A14YSMLYLJEMET,1.0,This product is no where near natural / organi...,2013-03-29,product near natural organic wish review purch...
3,31,B00005344V,Traditional Medicinals Organic Breathe Easy Se...,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",A2F488C4PLWGEI,5.0,If my wife drinks a cup of this tea when she f...,2014-03-23,wife drink cup tea feel attack come help avoid...
4,32,B00005344V,Traditional Medicinals Organic Breathe Easy Se...,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",AO1HXV7DWZZIR,5.0,I don't know about the medicinal aspects of th...,2014-02-06,know medicinal aspect tea flavor downright scr...
28001,77519,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A1WT3TVHANP7ZF,3.0,Hmmm. I really wanted to love this sweetener. ...,2014-07-22,hmmm want love sweetener half sugar half stevi...
28002,77520,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A3NEAETOSXDBOM,5.0,"I confess I have a sweet tooth, and love the t...",2014-06-30,confess sweet tooth love taste sugar recognize...
28003,77521,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",AD1ZOPB0BBEHB,4.0,"It has a little of the stevia aftertaste, but ...",2014-07-17,little stevia aftertaste fair compromise able ...
28004,77522,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A18ECVX2RJ7HUE,5.0,i love marinade for grilled flank steak or lon...,2014-05-30,love marinade grilled flank steak london broil...
28005,77523,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A2G04D4QZAXL15,3.0,I've been using Truvia (a form of stevia) on m...,2014-05-27,use truvia form stevia cereal greek yogurt yea...


In [11]:
# generating test history
test_user_history = (pd.DataFrame(test.groupby(['reviewerID'])['asin']
                                  .apply(list).reset_index()))

In [12]:
print(test_user_history)

                  reviewerID  \
0      A00177463W0XWB16A9O05   
1      A022899328A0QROR32DCT   
2      A068255029AHTHDXZURNU   
3      A06944662TFWOKKV4GJKX   
4             A1004703RC79J9   
...                      ...   
13274          AZWRZZAMX90VT   
13275          AZXKAH2DE6C8A   
13276          AZXON596A1VXC   
13277          AZYXC63SS008M   
13278          AZZ5ASC403N74   

                                                    asin  
0                               [B00474OR8G, B00BFM6OAW]  
1                                           [B00CMQDKES]  
2                                           [B001FA1K2G]  
3                                           [B000GFYRHG]  
4                                           [B003GTR8IO]  
...                                                  ...  
13274  [B0007R9L4M, B000CN7BMA, B001EQ5D1K, B002VT3GX...  
13275   [B000MAK41I, B004X8TJP2, B006H34CUS, B007W14RMM]  
13276                           [B001EO5S0I, B00271QQ7Q]  
13277                    

## Instantiate Memory-based Embedding CF (Item-based)

In [13]:
mem_ecf = cf.EmbeddedItemBasedCF(d2v)

In [14]:
# fit learning algorithm to training data
mem_ecf.fit(train)

100%|██████████████████████████████████████████████████████| 13397/13397 [00:00<00:00, 40798.12it/s]


In [15]:
# generate n-number of candidates items (200)
candidate_items = mem_ecf.test()

100%|███████████████████████████████████████████████████████| 13397/13397 [00:04<00:00, 2762.29it/s]


## Loop through N = {10, 25, 30, 45}

In [16]:
# generate item popularity
item_popularity = generate_item_popularity(train)

In [17]:
n_recommendations = {}
for n in [10, 25, 30, 45]:
    # retrieve the top-n items based on similarities
    top_ns = get_top_n(candidate_items, mem_ecf.user_rating_history, n)
    # evaluate how well the recommended items predicted the future purchases
    n_recommended_items = evaluate_recommendations(top_ns, test_user_history, item_popularity, n)
    # saving the n-value and recommended items
    n_recommendations[n] = (top_ns, n_recommended_items)

The MEM-ECF has an average recall@10: 0.03662, average novelty@10: 0.97241
The MEM-ECF has an average recall@25: 0.06371, average novelty@25: 0.97619
The MEM-ECF has an average recall@30: 0.07138, average novelty@30: 0.97686
The MEM-ECF has an average recall@45: 0.08744, average novelty@45: 0.97812


# Evaluate N-Recommendations

In [18]:
def retrieve_recommendations(train: pd.DataFrame, top_ns: dict):
    """
    """
    # generating a random user
    random_user = np.random.choice(list(train['reviewerID'].unique()), 1)[0]
    print(f"For user: {random_user}:")
    print(f"Purchase History:\n{train[train['reviewerID'] == random_user][['asin', 'title']]}")

    # find the recommendations
    print(f"\nRecommending:\n")
    recommendations = (train[train['asin']
                             .isin(top_ns[random_user])][['asin', 'title']]
                       .drop_duplicates(subset='asin')
                       .set_index('asin'))
    print(f"{recommendations.loc[top_ns[random_user]].reset_index()}")

## N=10

In [19]:
top_ns_10 = n_recommendations[10][0]

In [30]:
retrieve_recommendations(train, top_ns_10)

For user: A34057DJGD2E9A:
Purchase History:
             asin                                              title
24871  B001M23XK4  Mementa Tea, Organic Chinese White Tea, 100-Co...
35918  B004JR67XE  Galil Green Pitted Olive + 20% Extra Value Siz...

Recommending:

         asin                                              title
0  B000RI8NWW                  Yogi Tea Kombucha Green Tea 16 ea
1  B004VLV6F2  Bob's Red Mill Golden Couscous, 24-ounce (Pack...
2  B000QV8KNW                Legends of China White Tea 100 Bags
3  B001E5E1XC  Rishi Tea Organic Green Jasmine Tea Loose Tea,...
4  B00BY6NN30  Yogi Tea, Cinnamon Vanilla Healthy Skin, 16 Co...
5  B0046LHGQU        Roland Lentils, Red, 35.3 Ounce (Pack of 2)
6  B006M542TW  Non-GMO Project Verifed USA Grown 10 Lb Bean P...
7  B0047NSHBK  Allen's Popeye Spinach, 13.5000-Ounce (Pack of 6)
8  B004452P46  Mario Camacho Green Pitted Olives Seasoned wit...
9  B00AKKTYEI  Doritos Tortilla Chips, Nacho Cheese, 11 Ounce...


## N=25

In [21]:
top_ns_25 = n_recommendations[25][0]

In [22]:
retrieve_recommendations(train, top_ns_25)

For user: A5Y70Y025O34P:
Purchase History:
             asin                                              title
14358  B0011XKHY4             Biscoff Cookies, 8.8-Ounce (Pack of 5)
28291  B0029XLH4Y  Keurig, Coffee People, Jet Fuel, K-Cup Counts,...

Recommending:

          asin                                              title
0   B003M7TTYG  Green Mountain Coffee Revv, 22-Count K-Cups fo...
1   B0029XDZFS  Coffee People Wake Up Call K-Cup Portion Pack ...
2   B001D0IZBM                     Grocery &amp; Gourmet Food" />
3   B000J4IDO2  Coffee People K-Cups, Black Tiger Extra Bold, ...
4   B000TK6LBS  Caribou Coffee Caribou Blend, Keurig Single-Se...
5   B001CHFSU2                     Grocery &amp; Gourmet Food" />
6   B001CHFUDC  Coffee People Donut Shop Regular Medium Roast ...
7   B001ELL67A                     Grocery &amp; Gourmet Food" />
8   B004CZ1D8C  Green Mountain Coffee Keurig Nantucket Blend (...
9   B0029XDZDK  Coffee People Black Tiger K-Cup Portion Pack f...
10  B001

## N=30

In [23]:
top_ns_30 = n_recommendations[30][0]

In [24]:
retrieve_recommendations(train, top_ns_30)

For user: AS6SH2JM2YY3H:
Purchase History:
             asin                                              title
1996   B00099XMX4  Progresso Soup, Traditional, Homestyle Chicken...
9621   B000HGA782  Stretch Island Original Fruit Leather, Ripened...
12710  B000UUWECC  O.N.E. Coconut Water, 11.2-Ounce Aseptic Conta...
14128  B00110FINM  Yuban Breakfast Blend Ground Coffee, 12-Ounce ...
14573  B0012C55DW  mix 1 Protein &amp; Antioxidant Drink, Mix Ber...
17025  B001CEDW26  Wild Planet, Wild Pink Shrimp,  4 Ounce Cans (...
25922  B001SB8AZC  Wild Planet Wild Skipjack Tuna (Light Tuna), 5...
39014  B004YGQPAK  Ambrosoli-HoneesEnergy Plus Vitamin Honey (Pac...
40041  B005BPVB84  Eat. Think. Smile. Baked Nutrition Bar, Dark C...
40155  B005BPVCVU  Eat. Think. Smile. Baked Nutrition Bar, Orange...
44699  B007JFXWRC  Lipton Herbal Pyramid Tea Bags, Blackberry Van...
46456  B00ANX41O0  Belvita Cinnamon Brown Sugar Breakfast Biscuit...

Recommending:

          asin                              

## N=45

In [25]:
top_ns_45 = n_recommendations[45][0]

In [26]:
retrieve_recommendations(train, top_ns_45)

For user: A2IYSJV1O3DEMO:
Purchase History:
             asin                                              title
5240   B000EF3E5O  Crown Prince Natural Solid White Albacore Tuna...
8637   B000GFYRHG  Bigelow Constant Comment Tea 20-Count Boxes (P...
19459  B001E5E3JY  SweetLeaf Sweet Drops Liquid Stevia Sweetener,...

Recommending:

          asin                                              title
0   B001D0KG28  Timothy's World Tea, Chamomile Tea, 24-Count K...
1   B000GFYRK8  Bigelow Cozy Chamomile Herbal Tea Bags 20-Coun...
2   B00170NWEI  Rishi Tea, Organic Green Tea Sencha, 2.11-Ounc...
3   B000CMF18W  Yogi Teas St. John's Wort Blues Away, 16 Count...
4   B0019FOZC4  DoMatcha - Organic Ceremonial Matcha Powder, N...
5   B00142C0X8  Prince of Peace Organic Green Tea Jasmine, 100...
6   B0009F3PM6  Traditional Medicinals Organic Chamomile Herba...
7   B001QXYQFK  Bigelow Organic Rooibos with Asian Pear Tea, 1...
8   B001E5E38A  Uncle Lee's Tea Organic White Tea, Tea Bags, 1...
9   